# Steam Marketplace Analysis

#### Jacob Knispel (Team Lead), Nithin Perumal, Alec Tiefenthal, Matt Buckner

### Table of Contents

* [Executive Summary](http://localhost:8888/notebooks/ReportDraft.ipynb#Executive-Summary)
* [Introduction](http://localhost:8888/notebooks/ReportDraft.ipynb#Introduction)
* [Data Preprocessing](http://localhost:8888/notebooks/ReportDraft.ipynb#Data-Preprocessing)
* [Analysis](http://localhost:8888/notebooks/ReportDraft.ipynb#Analysis)
* [Results](http://localhost:8888/notebooks/ReportDraft.ipynb#Results)
* [Links](http://localhost:8888/notebooks/ReportDraft.ipynb#Links)
* [Disclaimer](http://localhost:8888/notebooks/ReportDraft.ipynb#Dislaimer)

## Executive Summary

In this project, we seek to examine the data provided by **Valve Software** pertaining to their online digital economy of in-game items in their game **Team Fortress 2** in the attempt to locate statistically interesting anomalies or trends that may not be immediately obvious to the outside observer.

## Introduction

## Data Preprocessing

In [ ]:
import requests;                                 # import package used to downlaod data from a web URL
import json;
import pandas as pd;
import seaborn as sns;
import matplotlib.pyplot as plt;                 # import all tools used to examine and plot the data
import matplotlib;
from bokeh.charts import Scatter, show
from bokeh.io import output_notebook
output_notebook()

from pandas.io.json import json_normalize

matplotlib.style.use('ggplot');
%matplotlib inline

The following lines utilize the API calls and key to download and store the data for examination

In [ ]:
with open('API Keys.txt', 'r') as keyfile:                              # This key was requested from Valve
    apiKeys=keyfile.read().split('\n')                                  # The specfic key utilized is included
   
url = 'http://backpack.tf/api/IGetPrices/v4/?key=' + apiKeys[0];        # web address of data stream

data_stream = requests.get(url,stream=True);                            # create an iterable data stream object

rec = data_stream.iter_lines().next().strip()                           # Necessary before reading data
data = json.loads(rec)                                                  # convert the json record to a python dictionary

In [ ]:
def byteify(input):
    if isinstance(input, dict):
        return {byteify(key): byteify(value)
                for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [byteify(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

data2 = byteify(data)

In [ ]:
df = json_normalize(data2['response'])

In [ ]:
#Define values provided by Valve.

raw_usd_value = df['raw_usd_value']
current_time = df['current_time']
success = df['success']
usd_currency_index = df['usd_currency_index']
usd_currency = df['usd_currency']

In [ ]:
#Import the item data

#THIS WILL TAKE >2 MINUTES TO RUN

dfItemFinal = pd.DataFrame(columns = ['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex', 'Currency', 'Value', 'Last_Update', 'Difference'])
df = json_normalize(data2['response'], 'items')
for itemName in df[0]:
    dfName = json_normalize(data2['response']['items'], [itemName])
    if(dfName.size != 2):
        raise Exception('More than 2 items for item' + itemName)
    if(dfName[0][0] != 'prices'):
        raise Exception('First row in %s is not prices' % (itemName))
    if(dfName[0][0] != 'prices'):
        raise Exception('Second row in %s is not defindex' % (itemName))
    defIndex = json_normalize(data2['response']['items'][itemName], ['defindex'])[0][0]
    dfPrices = json_normalize(data2['response']['items'][itemName], 'prices')
    for qualInt in dfPrices[0]:
        dfTrade = json_normalize(data2['response']['items'][itemName]['prices'], qualInt)
        
        for tradeable in dfTrade[0]:
            dfCraft = json_normalize(data2['response']['items'][itemName]['prices'][qualInt], tradeable)
        
            for craftable in dfCraft[0]:
                dfPriceIndex = json_normalize(data2['response']['items'][itemName]['prices'][qualInt][tradeable], craftable)
                
                for priceIndex in dfPriceIndex[0]:
                    dfFinalValues = json_normalize(data2['response']['items'][itemName]['prices'][qualInt][tradeable][craftable][priceIndex])
                    
                    currency = dfFinalValues['currency'][0]
                    difference = dfFinalValues['difference'][0]
                    last_update = dfFinalValues['last_update'][0]
                    value = dfFinalValues['value'][0]
                    dfItemFinal.loc[len(dfItemFinal)]=[itemName, defIndex, qualInt, tradeable, craftable, priceIndex, currency, value, last_update, difference]

In [ ]:
#Store the item data in an csv file for formatting and examinination

dfItemFinal.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex'])
#dfItemFinal = dfItemFinal.drop('Unnamed: 0', axis=1)
dfItemFinal.sort_index(inplace = True)
dfItemFinal.to_csv('itemOut.csv')

Once the data is stored in a .csv file, the following line prepares it for examination.
If a set of data has already been prepared, one may skip directly to this point to begin working with the dataset.

In [ ]:
#Move to our primary dataframe and access our item data

dfNew = pd.read_csv('itemOut.csv')
#dfNew = dfNew.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex'])
dfNew = dfNew.drop('Unnamed: 0', axis=1)
dfNew.sort_index(inplace = True)

## Analysis

In [ ]:
dfKeys = dfNew[dfNew['Currency'] == 'keys']
dfAllUnusuals = dfKeys[dfKeys['QualInt'] == 5]
dfUnusuals = dfAllUnusuals[dfAllUnusuals['Value'] < 200] # Remove obvious outliers
ax = dfUnusuals['Value'].plot(kind='hist', title='Distribution of Unusual Item Prices', bins=100)
ax.set_xlabel("Price in Keys")
ax.set_ylabel("Number of Items")

It's obvious from this graph that unusual items rarely sell for less than 5 or 6 keys, but most of them are sold for around 10 keys. This phenomenon can be explained very simply -- in order to get an unusual, a player must actually use a key to open something called a crate. They have a 1.00% chance to get an unusual item in return. 

For this reason, it wouldn't make sense for unusual items to sell for anywhere close to 1 key, because all unusuals, no matter how little people like them, are rarer than keys, which can be bought from the store via microtransactions (for $2.50)¶

In [ ]:
dfUnusuals = dfUnusuals[dfUnusuals['Value'] < 100] # Zoom
dfUnusuals = dfUnusuals[dfUnusuals['Value'] > 50] # Zooom!
ax = dfUnusuals['Value'].plot(kind='hist', title='Distribution of Unusual Item Prices (Zoomed!)', bins=30)
ax.set_xlabel("Price in Keys")
ax.set_ylabel("Number of Items")

It's interesting to note that, because the prices are defined freely by humans (by however much people decide to sell their items for), there is a great tendency for prices to fall on or near values that are multiples of 5. For example, looking at the range 70-80 keys, there are spikes at 70 keys, 75 keys, and 80 keys. 

- - -

Even the smaller spikes in-between the multiples of 5 can be explained -- these are at 73.33 and 76.66 from our range defined before. 

Why? Items on the TF2 market are often priced at values like '2.33 keys' or '1.66 refined metal' because these are more convenient to split that the obscure fractions the 'true price' usually lands on. 

In [ ]:
dfMetal = dfNew[dfNew['Currency'] == 'metal']
dfAllUniques = dfMetal[dfMetal['QualInt'] == 6]
dfUniques = dfAllUniques[dfAllUniques['Value'] < 15] # Remove obvious outliers
ax = dfUniques['Value'].plot(kind='hist', title='Distribution of Unique Item Prices', bins=30)
ax.set_xlabel("Price in Refined Metal")
ax.set_ylabel("Number of Items")

Similar to the previous graph, we can clearly see that unique items tend to sell for easily-divisible values. However, refined metal can actually be split in-game, always into 3 parts. Therefore, prices are almost always multiples of three -- the most common price for a unique item is 0.33 refined metal, which is equivalent to a single piece of a split refined metal. 

- - -

It's also interesting to note here how much cheaper unique items tend to be compared to unusuals. A unique item costs, on average, 2.60 refined metal. In comparison, the average unusual item costs a whopping 1392.24 refined metal! 

## Results

In [ ]:
print('Average unique item metal cost: %0.2f' % (dfAllUniques['Value'].mean()))
refinedMetalPerKey = dfAllUniques[dfAllUniques['Name'] == 'Mann Co. Supply Crate Key']['Value']
print('Average unusual item metal cost: %0.2f' % (dfAllUnusuals['Value'].mean() * refinedMetalPerKey))
dfAllUnusuals['Value'].mean()

## Links

[API keys.txt](https://github.com/NithinPerumal/SteamMarketAnalysis/blob/master/IPython%20Notebook/API%20Keys.txt "API keys.txt")

[Sample CSV dataset](https://github.com/NithinPerumal/SteamMarketAnalysis/blob/master/IPython%20Notebook/itemOut.csv " itemOut.csv")

[Known Quality Interger Meanings](https://github.com/NithinPerumal/SteamMarketAnalysis/blob/master/IPython%20Notebook/Known%20Quality%20Integer%20Meanings.txt "Known Quality Interger Meanings")

## Disclaimer

All data examined has been freely provided by Valve Software via their APIs for the purpose of allowing users to examine this publicly available information.

[Powered by Steam](http://steampowered.com)